In [26]:
suMin = float("inf")
suMax = -float("inf")
spMin = float("inf")
spMax = -float("inf")

In [27]:
def getData(uFd, pFd):
    global suMin
    global suMax
    global spMin
    global spMax
    print("working on file %s %s" % (uFd.name, pFd.name))
    x = []
    y = []
    while True:
        uL = uFd.readline()
        pL = pFd.readline()
        if not uL and pL: raise Exception("Different # lines")
        if uL and not pL: raise Exception("Different # lines")
        if not uL and not pL: break
            
        t1 = uL.split(",")[0]
        su = float(uL.split(",")[1])
        if su != float("inf") and su != -float("inf") and su < suMin: suMin = su
        if su != float("inf") and su != -float("inf") and su > suMax: suMax = su
            
        t2 = pL.split(",")[0]
        sp = float(pL.split(",")[1])
        if sp != float("inf") and sp != -float("inf") and sp < spMin: spMin = sp
        if sp != float("inf") and sp != -float("inf") and sp > spMax: spMax = su

            
        if t1 != t2: raise Exception("Inconsistent label, %s %s" % (pL, uL))
        
        x.append([su, sp]) #UA, PATH score
        if t1 == "m": y.append([1,0]) #one-hot
        elif t1 == "b": y.append([0,1]) #one-hot
        else: raise Exception("Invalid label, %s %s" % (pL, uL))

    return x, y

In [28]:
SEQ_LEN = 10
ua_data = '../hmm/UA_DATA/ua-5-fold/ua%d' % SEQ_LEN 
path_data = '../hmm/PATH_DATA/path-5-fold/path%d' % SEQ_LEN

folds = []
for f in range(1,6):
    
    uTrain = open("%s/ua.train%d.txt" % (ua_data, f), "r")
    pTrain = open("%s/path.train%d.txt" % (path_data, f), "r")
    uTest = open("%s/ua.test%d.txt" % (ua_data, f), "r")
    pTest = open("%s/path.test%d.txt" % (path_data, f), "r")
    
    xTrain, yTrain = getData(uTrain, pTrain)
    xTest, yTest = getData(uTest, pTest)
    folds.append([xTrain, yTrain, xTest, yTest])

    print("fold %d: " % f)
    print("train: ", len(xTrain), len(yTrain))
    print("test: ", len(xTest), len(yTest))

#replace INF
suMax += 1
suMin -= 1
spMax += 1
spMin -= 1
for xTrain, yTrain, xTest, yTest in folds:
    for i in range(len(xTrain)):
        if xTrain[i][0] == float("inf"): xTrain[i][0] = suMax
        if xTrain[i][0] == -float("inf"): xTrain[i][0] = suMin
        if xTrain[i][1] == float("inf"): xTrain[i][1] = spMax
        if xTrain[i][1] == -float("inf"): xTrain[i][1] = spMin 
    for i in range(len(xTest)):
        if xTest[i][0] == float("inf"): xTest[i][0] = suMax
        if xTest[i][0] == -float("inf"): xTest[i][0] = suMin
        if xTest[i][1] == float("inf"): xTest[i][1] = spMax
        if xTest[i][1] == -float("inf"): xTest[i][1] = spMin 

#normalized it to 0 to 1
for xTrain, yTrain, xTest, yTest in folds:
    for i in range(len(xTrain)):
        xTrain[i][0] = (xTrain[i][0] - suMin) / (suMax - suMin)
        xTrain[i][1] = (xTrain[i][1] - spMin) / (spMax - spMin)
    for i in range(len(xTest)):
        xTest[i][0] = (xTest[i][0] - suMin) / (suMax - suMin)
        xTest[i][1] = (xTest[i][1] - spMin) / (spMax - spMin)

print("done")

working on file ../hmm/UA_DATA/ua-5-fold/ua10/ua.train1.txt ../hmm/PATH_DATA/path-5-fold/path10/path.train1.txt
working on file ../hmm/UA_DATA/ua-5-fold/ua10/ua.test1.txt ../hmm/PATH_DATA/path-5-fold/path10/path.test1.txt
fold 1: 
train:  1913 1913
test:  459 459
working on file ../hmm/UA_DATA/ua-5-fold/ua10/ua.train2.txt ../hmm/PATH_DATA/path-5-fold/path10/path.train2.txt
working on file ../hmm/UA_DATA/ua-5-fold/ua10/ua.test2.txt ../hmm/PATH_DATA/path-5-fold/path10/path.test2.txt
fold 2: 
train:  1881 1881
test:  491 491
working on file ../hmm/UA_DATA/ua-5-fold/ua10/ua.train3.txt ../hmm/PATH_DATA/path-5-fold/path10/path.train3.txt
working on file ../hmm/UA_DATA/ua-5-fold/ua10/ua.test3.txt ../hmm/PATH_DATA/path-5-fold/path10/path.test3.txt
fold 3: 
train:  1900 1900
test:  472 472
working on file ../hmm/UA_DATA/ua-5-fold/ua10/ua.train4.txt ../hmm/PATH_DATA/path-5-fold/path10/path.train4.txt
working on file ../hmm/UA_DATA/ua-5-fold/ua10/ua.test4.txt ../hmm/PATH_DATA/path-5-fold/path10/p

In [40]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np

def getModel():
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

fold_acc = []
for fold_num in range(5):
    xTrain, yTrain, xTest, yTest = folds[fold_num]
    xTrain = np.array(xTrain)
    yTrain = np.array(yTrain)
    xTest = np.array(xTest)
    yTest = np.array(yTest)
    model = getModel()
    model.fit(xTrain, yTrain, epochs=20, batch_size=32, validation_data=(xTest, yTest))
    fold_acc.append(model.evaluate(xTest, yTest)[1])
    
for fold_num in range(5):
    print("fold %s has accuracy: %s" % (fold_num+1, fold_acc[fold_num]))
print("5 fold min %s, max %s, avg %s" % (min(fold_acc), max(fold_acc), sum(fold_acc)/5,))

Train on 1913 samples, validate on 459 samples
Epoch 1/20
1913/1913 [==============================] - 1s 481us/step - loss: 0.5384 - acc: 0.7841 - val_loss: 0.4820 - val_acc: 0.8170
Epoch 2/20
1913/1913 [==============================] - 0s 74us/step - loss: 0.5247 - acc: 0.7841 - val_loss: 0.4785 - val_acc: 0.8170
Epoch 3/20
1913/1913 [==============================] - 0s 75us/step - loss: 0.5234 - acc: 0.7841 - val_loss: 0.4765 - val_acc: 0.8170
Epoch 4/20
1913/1913 [==============================] - 0s 71us/step - loss: 0.5202 - acc: 0.7841 - val_loss: 0.4784 - val_acc: 0.8170
Epoch 5/20
1913/1913 [==============================] - 0s 71us/step - loss: 0.5168 - acc: 0.7841 - val_loss: 0.4715 - val_acc: 0.8170
Epoch 6/20
1913/1913 [==============================] - 0s 71us/step - loss: 0.5209 - acc: 0.7841 - val_loss: 0.4718 - val_acc: 0.8170
Epoch 7/20
1913/1913 [==============================] - 0s 72us/step - loss: 0.5092 - acc: 0.7841 - val_loss: 0.4748 - val_acc: 0.8170
Epoch 8

1900/1900 [==============================] - 0s 86us/step - loss: 0.5012 - acc: 0.7895 - val_loss: 0.5038 - val_acc: 0.7945
Epoch 20/20
472/472 [==============================] - 0s 34us/step
Train on 1961 samples, validate on 411 samples
Epoch 1/20
1961/1961 [==============================] - 1s 574us/step - loss: 0.5657 - acc: 0.7568 - val_loss: 0.3750 - val_acc: 0.9124
Epoch 2/20
1961/1961 [==============================] - 0s 84us/step - loss: 0.5265 - acc: 0.7649 - val_loss: 0.3599 - val_acc: 0.9124
Epoch 3/20
1961/1961 [==============================] - 0s 70us/step - loss: 0.5054 - acc: 0.7649 - val_loss: 0.3706 - val_acc: 0.9124
Epoch 4/20
1961/1961 [==============================] - 0s 71us/step - loss: 0.5006 - acc: 0.7649 - val_loss: 0.3634 - val_acc: 0.9124
Epoch 5/20
1961/1961 [==============================] - 0s 73us/step - loss: 0.5000 - acc: 0.7649 - val_loss: 0.3689 - val_acc: 0.9124
Epoch 6/20
1961/1961 [==============================] - 0s 71us/step - loss: 0.5013 -